In [ ]:
import requests
import pandas as pd
import time
from pathlib import Path

# Configuration
API_KEY = '' #Enter your own from core stack dashboard
BASE_URL = 'https://geoserver.core-stack.org/api/v1'
HEADERS = {'X-API-Key': API_KEY}

# Test locations in Western Ghats region
test_locations = [
    {'id': 1, 'name': 'Kodagu, Karnataka', 'lat': 12.50, 'lon': 75.50},
    {'id': 2, 'name': 'Karnataka-Kerala Border', 'lat': 13.50, 'lon': 75.50},
    {'id': 3, 'name': 'Central Karnataka', 'lat': 12.50, 'lon': 76.50},
    {'id': 4, 'name': 'Kerala-Tamil Nadu Border', 'lat': 9.50, 'lon': 77.50},
    {'id': 5, 'name': 'Wayanad-Idukki, Kerala', 'lat': 10.50, 'lon': 76.50},
    {'id': 6, 'name': 'Southern Western Ghats', 'lat': 8.50, 'lon': 77.50},
    {'id': 7, 'name': 'Coastal Karnataka', 'lat': 15.50, 'lon': 73.50},
    {'id': 8, 'name': 'Goa-Karnataka Border', 'lat': 16.50, 'lon': 73.50},
]

print(f"Testing {len(test_locations)} locations in Western Ghats region")
print("=" * 80)

Testing 8 locations in Western Ghats region


## Test 1: MWS ID Retrieval by Lat/Lon
Testing `/get_mwsid_by_latlon/` endpoint

In [4]:
def test_get_mws_by_latlon(lat, lon, location_name, max_retries=3):
    """Test MWS ID retrieval for a given lat/lon"""
    endpoint = f"{BASE_URL}/get_mwsid_by_latlon/"
    params = {'latitude': lat, 'longitude': lon}
    
    for attempt in range(max_retries):
        try:
            response = requests.get(endpoint, params=params, headers=HEADERS, timeout=30)
            
            result = {
                'location': location_name,
                'latitude': lat,
                'longitude': lon,
                'status_code': response.status_code,
                'success': response.status_code == 200,
                'response': None,
                'error': None
            }
            
            if response.status_code == 200:
                result['response'] = response.json()
            elif response.status_code == 404:
                result['error'] = 'Location not in coverage'
            else:
                result['error'] = f"HTTP {response.status_code}: {response.text[:200]}"
            
            return result
            
        except requests.exceptions.Timeout:
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
                continue
            result = {
                'location': location_name,
                'latitude': lat,
                'longitude': lon,
                'status_code': None,
                'success': False,
                'response': None,
                'error': 'Request timeout'
            }
            return result
            
        except Exception as e:
            result = {
                'location': location_name,
                'latitude': lat,
                'longitude': lon,
                'status_code': None,
                'success': False,
                'response': None,
                'error': f"Exception: {str(e)}"
            }
            return result

# Test all locations
mws_results = []

for loc in test_locations:
    print(f"Testing: {loc['name']} ({loc['lat']}, {loc['lon']})")
    result = test_get_mws_by_latlon(loc['lat'], loc['lon'], loc['name'])
    mws_results.append(result)
    
    if result['success']:
        print(f"  SUCCESS - MWS ID: {result['response']['uid']}")
        print(f"  Admin: {result['response']['District']}, {result['response']['State']}")
    else:
        print(f"  FAILED - {result['error']}")
    
    print()
    time.sleep(1)  # Rate limiting

mws_df = pd.DataFrame(mws_results)
print("\nSummary:")
print(f"Total locations tested: {len(mws_results)}")
print(f"Successful queries: {mws_df['success'].sum()}")
print(f"Failed queries: {(~mws_df['success']).sum()}")

Testing: Kodagu, Karnataka (12.5, 75.5)
  FAILED - Location not in coverage

Testing: Karnataka-Kerala Border (13.5, 75.5)
  FAILED - Location not in coverage

Testing: Central Karnataka (12.5, 76.5)
  FAILED - Location not in coverage

Testing: Kerala-Tamil Nadu Border (9.5, 77.5)
  SUCCESS - MWS ID: 2_21917
  Admin: VIRUDHUNAGAR, TAMIL NADU

Testing: Wayanad-Idukki, Kerala (10.5, 76.5)
  FAILED - Location not in coverage

Testing: Southern Western Ghats (8.5, 77.5)
  FAILED - Location not in coverage

Testing: Coastal Karnataka (15.5, 73.5)
  FAILED - Location not in coverage

Testing: Goa-Karnataka Border (16.5, 73.5)
  FAILED - Location not in coverage


Summary:
Total locations tested: 8
Successful queries: 1
Failed queries: 7


## Test 2: KYL Indicators Retrieval
Testing `/get_mws_kyl_indicators/` endpoint for locations with successful MWS ID retrieval

In [ ]:
def test_get_kyl_indicators(state, district, tehsil, mws_id, max_retries=3):
    """Test KYL indicators retrieval"""
    endpoint = f"{BASE_URL}/get_mws_kyl_indicators/"
    params = {
        'state': state,
        'district': district,
        'tehsil': tehsil,
        'mws_id': mws_id
    }
    
    for attempt in range(max_retries):
        try:
            response = requests.get(endpoint, params=params, headers=HEADERS, timeout=30)
            
            result = {
                'mws_id': mws_id,
                'state': state,
                'district': district,
                'status_code': response.status_code,
                'success': response.status_code == 200,
                'data': None,
                'error': None
            }
            
            if response.status_code == 200:
                result['data'] = response.json()
            elif response.status_code == 404:
                result['error'] = 'No KYL data available'
            else:
                result['error'] = f"HTTP {response.status_code}"
            
            return result
            
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
                continue
            result = {
                'mws_id': mws_id,
                'state': state,
                'district': district,
                'status_code': None,
                'success': False,
                'data': None,
                'error': str(e)
            }
            return result

# Test KYL indicators for successful MWS retrievals
kyl_results = []
successful_mws = mws_df[mws_df['success'] == True]

if len(successful_mws) > 0:
    print(f"Testing KYL indicators for {len(successful_mws)} locations with valid MWS IDs\n")
    
    for idx, row in successful_mws.iterrows():
        mws_data = row['response']
        print(f"Testing: {row['location']} - MWS {mws_data['uid']}")
        
        result = test_get_kyl_indicators(
            mws_data['State'],
            mws_data['District'],
            mws_data['Tehsil'],
            mws_data['uid']
        )
        kyl_results.append(result)
        
        if result['success']:
            print(f"  SUCCESS - Retrieved KYL indicators")
            if result['data'] and len(result['data']) > 0:
                indicators = result['data'][0]
                print(f"  Cropping intensity avg: {indicators.get('cropping_intensity_avg', 'N/A')}")
                print(f"  Avg precipitation: {indicators.get('avg_precipitation', 'N/A')} mm")
        else:
            print(f"  FAILED - {result['error']}")
        
        print()
        time.sleep(1)
    
    kyl_df = pd.DataFrame(kyl_results)
    print("\nSummary:")
    print(f"KYL queries attempted: {len(kyl_results)}")
    print(f"Successful: {kyl_df['success'].sum()}")
    print(f"Failed: {(~kyl_df['success']).sum()}")
else:
    print("No valid MWS IDs available for KYL indicator testing")

## Test 3: Water Balance Time Series
Testing `/get_mws_data/` endpoint for precipitation, ET, and runoff time series

In [ ]:
def test_get_mws_timeseries(state, district, tehsil, mws_id, max_retries=3):
    """Test time series data retrieval"""
    endpoint = f"{BASE_URL}/get_mws_data/"
    params = {
        'state': state,
        'district': district,
        'tehsil': tehsil,
        'mws_id': mws_id
    }
    
    for attempt in range(max_retries):
        try:
            response = requests.get(endpoint, params=params, headers=HEADERS, timeout=30)
            
            result = {
                'mws_id': mws_id,
                'state': state,
                'district': district,
                'status_code': response.status_code,
                'success': response.status_code == 200,
                'n_records': 0,
                'error': None
            }
            
            if response.status_code == 200:
                data = response.json()
                if 'time_series' in data:
                    result['n_records'] = len(data['time_series'])
            elif response.status_code == 404:
                result['error'] = 'No time series data available'
            else:
                result['error'] = f"HTTP {response.status_code}"
            
            return result
            
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
                continue
            result = {
                'mws_id': mws_id,
                'state': state,
                'district': district,
                'status_code': None,
                'success': False,
                'n_records': 0,
                'error': str(e)
            }
            return result

# Test time series for successful MWS retrievals
ts_results = []

if len(successful_mws) > 0:
    print(f"Testing time series data for {len(successful_mws)} locations\n")
    
    for idx, row in successful_mws.iterrows():
        mws_data = row['response']
        print(f"Testing: {row['location']} - MWS {mws_data['uid']}")
        
        result = test_get_mws_timeseries(
            mws_data['State'],
            mws_data['District'],
            mws_data['Tehsil'],
            mws_data['uid']
        )
        ts_results.append(result)
        
        if result['success']:
            print(f"  SUCCESS - Retrieved {result['n_records']} time series records")
        else:
            print(f"  FAILED - {result['error']}")
        
        print()
        time.sleep(1)
    
    ts_df = pd.DataFrame(ts_results)
    print("\nSummary:")
    print(f"Time series queries attempted: {len(ts_results)}")
    print(f"Successful: {ts_df['success'].sum()}")
    print(f"Failed: {(~ts_df['success']).sum()}")
else:
    print("No valid MWS IDs available for time series testing")

## Test 4: Administrative Details Validation
Testing `/get_admin_details_by_latlon/` endpoint

In [ ]:
def test_get_admin_details(lat, lon, location_name, max_retries=3):
    """Test administrative details retrieval"""
    endpoint = f"{BASE_URL}/get_admin_details_by_latlon/"
    params = {'latitude': lat, 'longitude': lon}
    
    for attempt in range(max_retries):
        try:
            response = requests.get(endpoint, params=params, headers=HEADERS, timeout=30)
            
            result = {
                'location': location_name,
                'latitude': lat,
                'longitude': lon,
                'status_code': response.status_code,
                'success': response.status_code == 200,
                'response': None,
                'error': None
            }
            
            if response.status_code == 200:
                result['response'] = response.json()
            elif response.status_code == 404:
                result['error'] = 'Location not in SOI boundary'
            else:
                result['error'] = f"HTTP {response.status_code}"
            
            return result
            
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
                continue
            result = {
                'location': location_name,
                'latitude': lat,
                'longitude': lon,
                'status_code': None,
                'success': False,
                'response': None,
                'error': str(e)
            }
            return result

# Test administrative details
admin_results = []

print("Testing administrative details retrieval\n")

for loc in test_locations:
    print(f"Testing: {loc['name']} ({loc['lat']}, {loc['lon']})")
    result = test_get_admin_details(loc['lat'], loc['lon'], loc['name'])
    admin_results.append(result)
    
    if result['success']:
        print(f"  SUCCESS - State: {result['response']['State']}")
        print(f"  District: {result['response']['District']}")
        print(f"  Tehsil: {result['response']['Tehsil']}")
    else:
        print(f"  FAILED - {result['error']}")
    
    print()
    time.sleep(1)

admin_df = pd.DataFrame(admin_results)
print("\nSummary:")
print(f"Admin queries attempted: {len(admin_results)}")
print(f"Successful: {admin_df['success'].sum()}")
print(f"Failed: {(~admin_df['success']).sum()}")

## Test 5: Active Locations Check
Testing `/get_active_locations/` endpoint to verify coverage area

In [ ]:
def test_get_active_locations(max_retries=3):
    """Test active locations retrieval"""
    endpoint = f"{BASE_URL}/get_active_locations/"
    
    for attempt in range(max_retries):
        try:
            response = requests.get(endpoint, headers=HEADERS, timeout=30)
            
            if response.status_code == 200:
                return {
                    'success': True,
                    'data': response.json(),
                    'error': None
                }
            else:
                return {
                    'success': False,
                    'data': None,
                    'error': f"HTTP {response.status_code}"
                }
                
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
                continue
            return {
                'success': False,
                'data': None,
                'error': str(e)
            }

print("Retrieving active locations from API\n")
active_locations = test_get_active_locations()

if active_locations['success']:
    print("SUCCESS - Active locations retrieved")
    data = active_locations['data']
    
    # Count states, districts, tehsils
    n_states = len(data.keys())
    n_districts = sum(len(state_data['districts']) for state_data in data.values())
    n_tehsils = sum(
        len(district_data['tehsils']) 
        for state_data in data.values() 
        for district_data in state_data['districts'].values()
    )
    
    print(f"\nCoverage Summary:")
    print(f"  States: {n_states}")
    print(f"  Districts: {n_districts}")
    print(f"  Tehsils: {n_tehsils}")
    
    print(f"\nStates covered:")
    for state in data.keys():
        print(f"  - {state.replace('_', ' ').title()}")
    
    # Check if any Western Ghats states are in coverage
    wg_states = ['karnataka', 'kerala', 'tamil_nadu', 'goa', 'maharashtra', 'gujarat']
    wg_states_in_coverage = [s for s in wg_states if s in data.keys()]
    
    print(f"\nWestern Ghats states in coverage: {len(wg_states_in_coverage)} of {len(wg_states)}")
    if wg_states_in_coverage:
        for state in wg_states_in_coverage:
            print(f"  - {state.replace('_', ' ').title()}")
    else:
        print("  None")
        
else:
    print(f"FAILED - {active_locations['error']}")

## Results Summary

In [ ]:
print("=" * 80)
print("COMPREHENSIVE TEST RESULTS")
print("=" * 80)
print()

# Create summary DataFrame
summary_data = {
    'Endpoint': [
        '/get_mwsid_by_latlon/',
        '/get_admin_details_by_latlon/',
        '/get_mws_kyl_indicators/',
        '/get_mws_data/',
        '/get_active_locations/'
    ],
    'Tested': [
        len(test_locations),
        len(test_locations),
        len(kyl_results) if 'kyl_results' in dir() and len(kyl_results) > 0 else 0,
        len(ts_results) if 'ts_results' in dir() and len(ts_results) > 0 else 0,
        1
    ],
    'Successful': [
        mws_df['success'].sum(),
        admin_df['success'].sum(),
        kyl_df['success'].sum() if 'kyl_df' in dir() else 0,
        ts_df['success'].sum() if 'ts_df' in dir() else 0,
        1 if active_locations['success'] else 0
    ],
    'Failed': [
        (~mws_df['success']).sum(),
        (~admin_df['success']).sum(),
        (~kyl_df['success']).sum() if 'kyl_df' in dir() else 0,
        (~ts_df['success']).sum() if 'ts_df' in dir() else 0,
        0 if active_locations['success'] else 1
    ]
}

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))
print()

# Geographic coverage assessment
print("GEOGRAPHIC COVERAGE ASSESSMENT:")
print("-" * 80)
print(f"Region tested: Western Ghats (8.5°N - 16.5°N, 73.5°E - 77.5°E)")
print(f"States: Karnataka, Kerala, Tamil Nadu, Goa")
print(f"Locations tested: {len(test_locations)}")
print(f"Locations with MWS data: {mws_df['success'].sum()} ({mws_df['success'].sum()/len(test_locations)*100:.1f}%)")
print()

if mws_df['success'].sum() == 0:
    print("FINDING: No Western Ghats locations returned valid MWS data")
    print("All queries returned 404 (Not Found) status")
    print()
    print("RECOMMENDATION:")
    print("  - Verify geographic coverage extends to southern Indian states")
    print("  - Check if Western Ghats coordinates are within SOI boundary definition")
    print("  - Confirm latitude/longitude parameter format expectations")
else:
    print("FINDING: Partial Western Ghats coverage detected")
    print("Successfully retrieved MWS data for some locations")

## Detailed Test Results Export

In [ ]:
# Save detailed results to CSV for further analysis
output_dir = Path('debug_output')
output_dir.mkdir(exist_ok=True)

# MWS test results
mws_df.to_csv(output_dir / 'mws_test_results.csv', index=False)
print(f"MWS test results saved: {output_dir / 'mws_test_results.csv'}")

# Admin test results
admin_df.to_csv(output_dir / 'admin_test_results.csv', index=False)
print(f"Admin test results saved: {output_dir / 'admin_test_results.csv'}")

# KYL test results (if any)
if 'kyl_df' in dir() and len(kyl_df) > 0:
    kyl_df.to_csv(output_dir / 'kyl_test_results.csv', index=False)
    print(f"KYL test results saved: {output_dir / 'kyl_test_results.csv'}")

# Time series test results (if any)
if 'ts_df' in dir() and len(ts_df) > 0:
    ts_df.to_csv(output_dir / 'timeseries_test_results.csv', index=False)
    print(f"Time series test results saved: {output_dir / 'timeseries_test_results.csv'}")

# Summary
summary_df.to_csv(output_dir / 'test_summary.csv', index=False)
print(f"Test summary saved: {output_dir / 'test_summary.csv'}")

print(f"\nAll test results exported to {output_dir}/")

## API Configuration Details

**API Key Used:** x0bXxURa.B9Qgfxd0aKxxJ8GIDHA5FCSIAc52hFgg

**Request Parameters:**
- Timeout: 30 seconds
- Max retries: 3 attempts with exponential backoff
- Rate limiting: 1 second delay between requests

**Expected vs. Actual:**
- Expected: Western Ghats region coverage (major biodiversity hotspot)
- Actual: No coverage detected for tested coordinates

**Next Steps for API Team:**
1. Verify geographic boundaries of current coverage
2. Confirm whether southern Indian states are planned for inclusion
3. Validate coordinate system compatibility (WGS84 assumed)
4. Review 404 response handling for locations outside coverage